In [10]:
import transformers
import datasets
import sklearn
import pandas as pd
import numpy as np

### Datensatz laden

In [22]:
# Datensatz laden

file_off = "..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\HateSpeechDe_HATE_GruppeDetail.json"
file_on = "../data_bert/HateSpeechDe_HATE_Handlung.json"
daten = pd.read_json(file_off)
daten = daten.set_index(keys="corpus_id")

print(daten)

                              label  \
corpus_id                             
1112521               [KeineGruppe]   
1114995               [KeineGruppe]   
1110545               [KeineGruppe]   
1114326    [Politische Einstellung]   
4112169               [KeineGruppe]   
...                             ...   
1223336               [KeineGruppe]   
1221963               [KeineGruppe]   
2220834    [Politische Einstellung]   
1220580           [Anderes Merkmal]   
2222357               [KeineGruppe]   

                                                       tweet  
corpus_id                                                     
1112521    @user @user @user Weitaus schlimmer. Heute ist...  
1114995     Das Deutsche Kaiserreich soll wieder auferstehen  
1110545                     Die BRD ist eine einzige Schande  
1114326    @user @user Die Grünen....besser kann man das ...  
4112169    @user @user Scheiss deutsche Politiker! Mehr g...  
...                                                

### Annotationslabels encodieren

In [12]:
from sklearn.preprocessing import MultiLabelBinarizer

def encode(labels):
    """
    Input: labels = Liste der Annotationslabels für den Datensatz,
            z.B. ["KEINE", "KEINE", "VVH", "KEINE", ...] oder
                 [["KeineGruppe"], ["Politische Einstellung", "Geschlecht"], ["KeineGruppe"], ...]
    Output:
            namen = Liste der Klassenlabels in korrekter Reihenfolge;
                    eine der Listen in label_namen
            labels_encoded = Liste der Annotationslabels im binären Format
                    die Probleme stellen immer die Frage "Ist dieses Phänomen vorhanden?" - Ja/Nein
                    dann im binären Format: Ja = 1, Nein = 0
    """

    # Mögliche Klassen
    labels_vvh = ["KEINE", "VVH"]
    labels_gruppe = ["KeineGruppe", "Gruppe"]
    labels_handlung = ["KeineHandlung", "Handlung"]
    labels_gruppe_det = ["KeineGruppe", "Nationalität", 'ethnische Herkunft / "Rasse"', "Religion / Weltanschauung",
        "Politische Einstellung", "Geschlecht", "Anderes Merkmal"]
    labels_handlung_det = ["KeineHandlung", "Aufstachelung zu Hass", "Aufforderung zu Gewalt- oder Willkürmaßnahmen", "Angriff der Menschenwürde"]
    label_namen = [labels_vvh, labels_gruppe, labels_handlung, labels_gruppe_det, labels_handlung_det]


    # Klassifizierungsproblem, also das Set der vorhandenen Labels, ermitteln

    # Fall 1: Strings (binäre Klassen)
    namen = []
    if type(labels[0]) == str:
        labels_flat = set(labels)
    # Fall 2: Listen (mehrere Klassen)
    else:
        # Summe über die Sets aller Listen
        labels_flat = set([label for entry in labels for label in entry])
    for i in label_namen:
        if set(i) == labels_flat: namen = i


    # Labels transformieren

    # Fall 1: 2 Klassen
    eins = ["NEG", "VVH", "Gruppe", "Handlung"]
    if len(namen) == 2:
        labels_eins = np.array(list(map(lambda x: 1 if x in eins else 0, list(labels))))
        return (labels_eins, namen)

    # Fall 2: mehrere Klassen
    else:
        binarizer = MultiLabelBinarizer(classes = namen)
        label_array = binarizer.fit_transform(labels)

        # leere Matrix der Form Einträge x Labelnamen
        #label_array = [[0 for i in range(len(namen))] for j in range(len(labels))]
        # Matrix befüllen
        #for i, label_liste in enumerate(labels):
        #    for j, name in enumerate(namen):
        #        if name in label_liste:
        #            label_array[i][j] = 1
        # die i-te Spalte Ndarrays label_array ist jeweils die Annotation für die i-te Klasse
        # also bei namen = handlung_det, ist label_array[:, 0] die Annotation für "KeineHandlung"
        #label_array = [np.array(liste) for liste in label_array]
        #label_array = np.array(label_array)
        return (label_array, namen)

### Stratifizierter Train/Test-Split

In [13]:
from sklearn.model_selection import train_test_split
from skmultilearn.model_selection import iterative_train_test_split,IterativeStratification
from skmultilearn.model_selection.measures import get_combination_wise_output_matrix
from collections import Counter

def train_test_split_multilabel(daten, ziele, test_size=0.3):
    """TODO Beschreibung
    """

    # Input daten als Input für den Multilabel-Stratifizierer (http://scikit.ml/stratification.html) vorbereiten:
    # Format ndarray (beispiele) x ndarray (features)
    # da allerdings die Features erst nach dem Train/Test-Split berechnet werden,
    # werden stattdessen die Korpus-IDs übergeben, anhand derer dann die Tweets zugeordnet werden
    #X, y  = daten.index, ziele
    X, y = np.array([np.array([entry, ]) for entry in daten.index]), ziele
    #X = np.array(X)

    # Datenanalyse: Anzahl der Einträge pro Klasse / Klassenkombination
    # Counter(combination for row in get_combination_wise_output_matrix(y, order=1) for combination in row))

    X_train, y_train, X_test, y_test = iterative_train_test_split(X, y, test_size = test_size) # Multilabel

    # Tweets anhand der IDs dem Train/Test-Split zuordnen
    X_train_tweets = [daten.loc[index[0]]["tweet"] for index in X_train]
    X_test_tweets =  [daten.loc[index[0]]["tweet"] for index in X_test]

    return X_train_tweets, X_test_tweets, y_train, y_test


def split(daten, ziele, labels):
    """Train/Test-Split 
    Input: Datensatz (Pandas Dataframe mit Index und Spalte "tweets"),
           ziele (Zielannotation im binären Format)
    Output: X_train, y_train, X_test, y_test
            nach stratifiziertem Train/Test-Split, Preprocessing, Merkmalsauswahl und Normalisierung
    """

    # Stratifizierter Train/Test-Split
    if len(labels) == 2:
        X_train, X_test, y_train, y_test = train_test_split(daten["tweet"], ziele, test_size=0.33, random_state=36, stratify=ziele)
    else:
        X_train, X_test, y_train, y_test = train_test_split_multilabel(daten, ziele, test_size=0.3)

    return (X_train, y_train), (X_test, y_test)    


### Tokenizer für das Preprocessing laden

In [14]:
from transformers import AutoTokenizer

# Für das Preprocessing mit Sklearn: kein Padding
tokenizer = AutoTokenizer.from_pretrained('deepset/gelectra-large', truncation=True, padding=False)

def bert_tokenize(inputs):
    """Einen String mit BERT tokenisieren
    Output: Liste von Tokens"""
    token_ids = tokenizer(inputs)
    tokens = tokenizer.convert_ids_to_tokens(token_ids["input_ids"])
    return tokens

# Für das Preprocessing für BERT: Padding, Output: Pytorch tensons
tokenizer_bert = AutoTokenizer.from_pretrained('deepset/gelectra-large', padding=True, truncation=True, return_tensors="pt")

def preprocess(data):
    return tokenizer_bert(data["text"])

In [90]:
tweet1 = "@user Das meiste geht eh wieder für die Asylanten drauf 🤮"
tweet2 = "@user Die ganze Bande muss weg!"
tweet1 = tweet1.lower()
tweet2 = tweet2.lower()
print(tweet2)
tweettoks = tokenizer(tweet2)
print(tokenizer.convert_ids_to_tokens(tweettoks["input_ids"]))

@user die ganze bande muss weg!
['[CLS]', '@', 'use', '##r', 'die', 'ganze', 'ba', '##nde', 'muss', 'weg', '!', '[SEP]']


### Evaluationsmetriken vorbereiten

In [16]:
from sklearn import metrics
from sklearn.metrics import PrecisionRecallDisplay, precision_recall_curve, multilabel_confusion_matrix, matthews_corrcoef

def eval(y_test, predicted, labels):
    '''TODO
    '''
    # Evaluation
    # TODO: restliche Metriken ergänzen
    evaluation = dict() # Precision, Recall, Accuracy, F1, MCC, Confusion Matrix

    # metrics.classification_report(y_test, predicted, labels)
    evaluation["confusion"] = metrics.confusion_matrix(y_test, predicted)
    evaluation["mcc"] = metrics.matthews_corrcoef(y_test, predicted)

    return (predicted, evaluation)

### Klassifikationsmethode Nr.1: Logistischer Regression

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

def features_tfidf(X_train, X_test):
    '''TODO Beschreibung
    Preprocessing und Merkmalsauswahl
    Output: X_train, y_train, X_test, y_test
            nach stratifiziertem Train/Test-Split, Preprocessing, Merkmalsauswahl und Normalisierung
    '''

    # Preprocessing und Merkmalsauswahl
    # weitere mögliche Parameter: strip_accents = unicode, max_features=100, min_df = 2, sublinear_tf = True (replace tf with 1 + log(tf))
    vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), lowercase=True, tokenizer=bert_tokenize, stop_words=["[SEP]", "[CLS]"])
    X_train_feat = vectorizer.fit_transform(X_train)
    X_test_feat = vectorizer.transform(X_test)

    # Normalisierung
    max_abs_scaler = preprocessing.MaxAbsScaler()
    X_train_maxabs = max_abs_scaler.fit_transform(X_train_feat)
    X_test_maxabs = max_abs_scaler.transform(X_test_feat)

    return X_train_maxabs, X_test_maxabs


def pipeline(daten, ziele, labels):
    '''TODO Beschreibung
    für zwei Klassen
    '''
    train, test = split(daten, ziele, labels)
    train[0], test[0] = features_tfidf(train[0], test[0])
    
    X_train, y_train = train
    X_test, y_test = test

    # Training
    model = LogisticRegression()
    model.fit(X_train, y_train)    

    predicted = model.predict(X_test)
    evaluation = eval(y_test, predicted, labels)
    return evaluation

def multilabel_pipeline(daten, ziele, labels):
    """TODO Beschreibung
    """
    # 1. Klassifikationspipeline (train_eval) für jede Klasse einmal laufen lassen,
    #    Evaluationsergebnisse sammeln
    train, test = split(daten, ziele, labels)
    train_feat, test_feat = features_tfidf(train[0], test[0]) 
    X_train, y_train = train_feat, train[1]
    X_test, y_test = test_feat, test[1]

    # sammelt alle Tupel (predicted, evaluation)
    # für die i-te Klasse an der jeweils i-ten Stelle
    ergebnisse_gsmmlt = []
    predicted_gsmmlt = []
    for i, label in enumerate(labels):
        # für jede Klasse angepasste Labelliste
        labels_i = ["Keine", label]

        train_i = (X_train, y_train[:, i])
        test_i = (X_test, y_test[:, i])

        # Training
        model = LogisticRegression()
        model.fit(X=train_i[0], y=train_i[1])    

        predicted = model.predict(test_i[0])
        evaluation = eval(test_i[1], predicted, labels_i)
        
        ergebnisse_gsmmlt.append(evaluation)
        predicted_gsmmlt.append(predicted)

    # 2. Gesammelte Ergebnisse evaluieren
    # gesammeltes Predict
    # Matrix im Format Einträge (Reihen) x Klassen (Spalten)
    predicted_gsmmlt = np.array(predicted_gsmmlt).transpose()
    confusion = multilabel_confusion_matrix(y_test, predicted_gsmmlt)#, labels=labels)

    return confusion

### Klassifikationsmethode Nr. 2: Transfer Learning

In [18]:
from datasets import load_dataset, Dataset
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding


data_collator = DataCollatorWithPadding(tokenizer=tokenizer_bert)

def pipeline_bert(daten, ziele, labels):
    '''TODO Beschreibung
    Klassifikation für 2 Klassen mit BERT
    
    '''

    train, test = split(daten, ziele, labels)

    X_train, y_train = train
    X_test, y_test = test

    train = {"text": X_train, "labels": y_train}
    test = {"text": X_test, "labels": y_test}

    train_dataset = Dataset.from_dict(train)
    test_dataset = Dataset.from_dict(test)

    train_tokenized = train_dataset.map(preprocess, batched=True)
    test_tokenized = test_dataset.map(preprocess, batched=True)
    train_tokenized = train_tokenized.remove_columns("text")
    test_tokenized = test_tokenized.remove_columns("text")

    # Fine-Tuning
    model = AutoModelForSequenceClassification.from_pretrained('deepset/gelectra-large', num_labels=2)

    training_args = TrainingArguments(
        output_dir="./results",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=10,
        weight_decay=0.01,
        evaluation_strategy = 'no'
    )


    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_tokenized,
        eval_dataset=test_tokenized,
        tokenizer=tokenizer_bert,
        data_collator=data_collator,
    )

    trainer.train()

    pred = trainer.predict(test_dataset=test_tokenized)
    predicted = np.argmax(pred[0], axis=-1)    

    evaluation = eval(y_test, predicted, labels)
    
    return evaluation


def multilabel_pipeline_bert(daten, ziele, labels):
    '''
    Klassifizierung mit BERT für mehr als zwei Klassen
    TODO Beschreibung
    '''
    
    train, test = split(daten, ziele, labels)
    X_train, y_train = train
    X_test, y_test = test

    train = {"text": X_train, "labels": y_train}
    test = {"text": X_test, "labels": y_test}
    train_dataset = Dataset.from_dict(train)
    test_dataset = Dataset.from_dict(test)

    train_tokenized = train_dataset.map(preprocess, batched=True)
    test_tokenized = test_dataset.map(preprocess, batched=True)
    train_tokenized = train_tokenized.remove_columns("text")
    test_tokenized = test_tokenized.remove_columns("text")

    # sammelt alle Tupel (predicted, evaluation)
    # für die i-te Klasse an der jeweils i-ten Stelle
    ergebnisse_gsmmlt = []
    predicted_gsmmlt = []
    for i, label in enumerate(labels):
        # für jede Klasse angepasste Labelliste
        labels_i = ["Keine", label]

        # i-te Spalte in der Labelmatrix auswählen, dabei den letzten Eintrag ersetzen
        train_tokenized = train_tokenized.remove_columns("labels")
        test_tokenized = test_tokenized.remove_columns("labels")
        train_tokenized = train_tokenized.add_column("labels", y_train[:, i])
        test_tokenized = test_tokenized.add_column("labels", y_test[:, i])

        # Fine-Tuning
        model = AutoModelForSequenceClassification.from_pretrained('bert-base-german-cased', num_labels=2)

        training_args = TrainingArguments(
            output_dir="./results",
            learning_rate=2e-5,
            per_device_train_batch_size=16,
            per_device_eval_batch_size=16,
            num_train_epochs=1,
            weight_decay=0.01,
            evaluation_strategy = 'no'
        )

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_tokenized,
            eval_dataset=test_tokenized,
            tokenizer=tokenizer_bert,
            data_collator=data_collator,
        )

        trainer.train()

        pred = trainer.predict(test_dataset=test_tokenized)
        predicted = np.argmax(pred[0], axis=-1)    

        evaluation = eval(y_test[:, i], predicted, labels)

        ergebnisse_gsmmlt.append(evaluation)
        predicted_gsmmlt.append(predicted)


    # 2. Gesammelte Ergebnisse evaluieren
    # gesammeltes Predict
    # Matrix im Format Einträge (Reihen) x Klassen (Spalten)
    predicted_gsmmlt = np.array(predicted_gsmmlt).transpose()
    confusion = multilabel_confusion_matrix(y_test, predicted_gsmmlt)#, labels=labels)

    return confusion

### Pipeline für eine oder mehrere Klassen laufen lassen

In [23]:
labels_encoded, label_namen = encode(list(daten["label"]))

print(multilabel_pipeline(daten, labels_encoded, label_namen))

#print(pipeline_bert(daten, labels_encoded, label_namen))


#multilabel_pipeline(daten, labels_encoded, label_namen)

c:\Users\cbi\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['##p', '##s', '[', ']', 'cl', 'se'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


TypeError: fit() missing 1 required positional argument: 'y'